<a href="https://colab.research.google.com/github/tutuponnekanty/machinelearning/blob/main/k_Best.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Importing Required Libraries and Modules**

In [ ]:
#importing required ML - Python Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt

In [ ]:
#Rose-Pine-Dawn Matplotlib Module for Visual enhancement and 3D-graphical compatability with seaborn
!wget https://raw.githubusercontent.com/h4pZ/rose-pine-matplotlib/main/themes/rose-pine-dawn.mplstyle -qP /tmp
plt.style.use("/tmp/rose-pine-dawn.mplstyle")

In [ ]:
#ignoring the warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
#downloading the datset and loading it into the VM
!wget -q https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip
!unzip -qq UCI\ HAR\ Dataset.zip

In [ ]:
#loading files into the colab
import os
for dirname, _, filenames in os.walk('/content'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#checking the dataset
df_samp = pd.read_csv("/content/UCI HAR Dataset/train/Inertial Signals/body_acc_x_train.txt", sep="\s+", header=None)
df_samp.head()

In [ ]:
df_samp_test = pd.read_csv("/content/UCI HAR Dataset/test/Inertial Signals/body_acc_x_test.txt", sep="\s+", header=None)

df_samp_test.head()

# **2. Accumilating the Dataset into one Array**

In [ ]:

f = open("/content/UCI HAR Dataset/README.txt", "r", encoding="latin-1")
print(f.read())
f.close()

In [ ]:
def load_file(filepath):
    df = pd.read_csv(filepath, header=None, delim_whitespace=True)
    return df.values

In [ ]:
def load_group(files, prefix=''):
    loaded = list()
    for f in files:
        data = load_file(prefix + f)
        loaded.append(data)
    loaded = np.dstack(loaded)
    return loaded

In [ ]:
def load_dataset_group(group, prefix='/content/UCI HAR Dataset/'):
    filepath = prefix + group + '/Inertial Signals/'
    files = list()
    files += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    files += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    files += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    X = load_group(files, filepath)
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y

In [ ]:
def load_dataset(prefix='/content/UCI HAR Dataset/'):
    X_train, y_train = load_dataset_group('train', prefix)
    X_test, y_test = load_dataset_group('test', prefix)
    print(f"""Dataset loaded.
Training Set:
X_train {X_train.shape} y_train {y_train.shape}
Test Set:
X_test {X_test.shape} y_test {y_test.shape}""")
    return X_train, y_train, X_test, y_test

In [ ]:
X_train, y_train, X_test, y_test = load_dataset()

In [ ]:
activity = {
        1: 'Walking',
        2: 'Walking Upstairs',
        3: 'Walking Downstairs',
        4: 'Sitting',
        5: 'Standing',
        6: 'Laying'}
def activities(obs):
    return activity[int(y_train[obs])]

In [ ]:
def features(feature):
    f={"Body acceleration": 0, "Gyro": 1, "Total acceleration": 2}
    return f[feature]

In [ ]:
sample=[777, 666, 818, 0,6666,66]
[activity[int(y_train[i])] for i in sample]

# **3. EDA**

In [ ]:
def get_values(y_values, T, N, f_s, sample_rate):
    y_values = y_values
    x_values = [sample_rate * kk for kk in range(0,len(y_values))]
    return x_values, y_values

In [ ]:
def signal_viz(obs):
    N = 128  # number of timesteps
    f_s = 50  # overlapped percentage
    t_n = 2.56  # time
    T = t_n / N
    sample_rate = 1 / f_s

    labels = ['x-component', 'y-component', 'z-component']
    colors = ['#eb6f92', '#9ccfd8', '#f6c177']  # Soft Rose Pine palette (red, cyan, gold)
    suptitle = "Different signals for the activity: {}"
    xlabel = 'Time [sec]'
    ylabel = 'Amplitude'
    axtitles = ['Body acceleration', 'Gyro', 'Total acceleration']
    activity_name = activities(obs)

    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(24, 8))
    fig.patch.set_facecolor('#faf4ed')  # Rose Pine Dawn background

    for comp_no in range(0, 9):
        col_no = comp_no // 3
        plot_no = comp_no % 3
        color = colors[plot_no]
        label = labels[plot_no]
        axtitle = axtitles[col_no]

        ax = axes[col_no]
        ax.set_title(axtitle, fontsize=16)
        ax.set_xlabel(xlabel, fontsize=14)
        ax.set_facecolor('#faf4ed')  # Light background for axes
        ax.grid(True, color='#e0def4', linestyle='--', linewidth=0.5, alpha=0.7)

        if col_no == 0:
            ax.set_ylabel(ylabel, fontsize=14)

        signal_component = X_train[obs][:, comp_no]
        x_values, y_values = get_values(signal_component, T, N, f_s, sample_rate)
        ax.plot(x_values, y_values, linestyle='-', color=color, label=label, linewidth=1.8)

        if col_no == 2:
            ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=12)

    fig.suptitle(suptitle.format(activity_name), fontsize=20, weight='bold')
    plt.tight_layout()
    plt.subplots_adjust(top=0.88, hspace=0.4)
    plt.show()


In [ ]:
for i in sample:
    signal_viz(i)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl

# Apply Rose Pine Dawn styling
sns.set_theme(style="whitegrid", font_scale=1.8)

def signal_3dviz(obs, feature):
    # Assuming `activities()` returns the activity name and `features()` gives the index
    activity_name = activities(obs)
    i = features(feature)

    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(111, projection="3d")
    fig.patch.set_facecolor('#faf4ed')  # Background to match style

    # Extract 3D components
    x = X_train[obs][:, i * 3 + 0]
    y = X_train[obs][:, i * 3 + 1]
    z = X_train[obs][:, i * 3 + 2]

    # Use a soft color from Rose Pine palette
    ax.plot(x, y, z, color="#eb6f92", label=feature, linewidth=2.5)

    # Labels and title
    ax.set_title(activity_name, fontsize=20, weight='bold')
    ax.set_xlabel("X", fontsize=16)
    ax.set_ylabel("Y", fontsize=16)
    ax.set_zlabel("Z", fontsize=16)

    # Make axes planes transparent
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False

    # Remove grid lines for a minimal look
    ax.grid(False)

    # Style ticks
    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    ax.tick_params(axis='z', labelsize=12)

    # Legend
    ax.legend(fontsize=14)

    plt.show()


In [ ]:
for i in sample:
    signal_3dviz(i, "Body acceleration")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl

# Apply Rose Pine Dawn styling
sns.set_theme(style="whitegrid", font_scale=1.8)

def signal_3dviz(obs, feature):
    # Assuming `activities()` returns the activity name and `features()` gives the index
    activity_name = activities(obs)
    i = features(feature)

    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(111, projection="3d")
    fig.patch.set_facecolor('#faf4ed')  # Background to match style

    # Extract 3D components
    x = X_train[obs][:, i * 3 + 0]
    y = X_train[obs][:, i * 3 + 1]
    z = X_train[obs][:, i * 3 + 2]

    # Use a soft color from Rose Pine palette
    ax.plot(x, y, z, color="#eb6f92", label=feature, linewidth=2.5)

    # Labels and title
    ax.set_title(activity_name, fontsize=20, weight='bold')
    ax.set_xlabel("X", fontsize=16)
    ax.set_ylabel("Y", fontsize=16)
    ax.set_zlabel("Z", fontsize=16)

    # Make axes planes transparent
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False

    # Remove grid lines for a minimal look
    ax.grid(False)

    # Style ticks
    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    ax.tick_params(axis='z', labelsize=12)

    # Legend
    ax.legend(fontsize=14)

    plt.show()


In [ ]:
for i in sample:
    signal_3dviz(i, "Gyro")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Apply Rose Pine Dawn style globally
sns.set_theme(style="white", font_scale=1.8)

def distance_viz(obs, feature):
    graph_name = "graph/distance {} {}.png"
    activity_name = activities(obs)

    i = features(feature)

    fig = plt.figure(figsize=(10, 6))
    fig.patch.set_facecolor('#faf4ed')  # Rose Pine Dawn background

    # Extract components
    x = X_train[obs][:, i * 3 + 0]
    y = X_train[obs][:, i * 3 + 1]
    z = X_train[obs][:, i * 3 + 2]

    # Calculate Euclidean distance
    distance = (x**2 + y**2 + z**2)**0.5

    # Plot with soft, warm Rose Pine color
    plt.plot(distance, label=feature, color="#eb6f92", linewidth=2.2)

    # Labels and title
    plt.title(activity_name, fontsize=20, weight='bold')
    plt.xlabel("Timesteps", fontsize=16)
    plt.ylabel("Distance", fontsize=16)

    # Legend
    plt.legend(fontsize=14)

    # Ticks styling
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    # Optional: Remove top and right spines for minimalism
    sns.despine()

    plt.tight_layout()
    plt.show()


In [ ]:
for i in sample:
    distance_viz(i, "Body acceleration")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Apply Rose Pine Dawn style
sns.set_theme(style="whitegrid", font_scale=2.5)

def y_graph():
    # Combine labels and map to activity names
    y = pd.DataFrame(np.concatenate((y_train, y_test)), columns=["Activity"])
    y["Activity"] = y["Activity"].map(activity)

    # Create the figure
    fig, ax = plt.subplots(figsize=(36, 14))
    fig.patch.set_facecolor('#faf4ed')  # Rose Pine background

    # Plot with a soft color palette
    sns.countplot(data=y, y="Activity", ax=ax, palette=["#eb6f92", "#9ccfd8", "#f6c177", "#31748f", "#c4a7e7", "#ea9a97", "#f6c177"])

    # Titles and labels
    ax.set_title("Observations by Activity", fontsize=28, weight='bold')
    ax.set_xlabel("Count", fontsize=22)
    ax.set_ylabel("Activity", fontsize=22)

    # Customize tick sizes
    ax.tick_params(axis='x', labelsize=18)
    ax.tick_params(axis='y', labelsize=18)

    # Remove spines for minimal look
    sns.despine(left=True, bottom=True)

    # Tight layout
    plt.tight_layout()
    plt.show()


In [ ]:
y_graph()

# **4. Filter Based Approach (main) - Fischer's Score with Selective Feature Selection**

In [ ]:
df = pd.read_csv('train.csv')
df.head()

In [ ]:
df['Activity'].value_counts()

In [ ]:
df.shape

In [ ]:
df.describe().T

In [ ]:
df.info()


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Separate features and target
X = df.drop('Activity', axis=1)
y = df['Activity']

# Encode target labels
le = LabelEncoder()
y = le.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
#log-reg
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

# Make predictions on the test set
y_pred = log_reg.predict(X_test)

# Calculate and print accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy:", accuracy)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf_classifier.predict(X_test)

# Calculate and print accuracy score for Random Forest
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Random Forest Test accuracy:", accuracy_rf)


In [ ]:
def get_duplicate_columns(df):

    duplicate_columns = {}
    seen_columns = {}

    for column in df.columns:
        current_column = df[column]

        # Convert column data to bytes
        try:
            current_column_hash = current_column.values.tobytes()
        except AttributeError:
            current_column_hash = current_column.to_string().encode()

        if current_column_hash in seen_columns:
            if seen_columns[current_column_hash] in duplicate_columns:
                duplicate_columns[seen_columns[current_column_hash]].append(column)
            else:
                duplicate_columns[seen_columns[current_column_hash]] = [column]
        else:
            seen_columns[current_column_hash] = column

    return duplicate_columns

In [ ]:
duplicate_columns = get_duplicate_columns(X_train)

In [ ]:
duplicate_columns

In [ ]:
X_train[['tBodyAccMag-mean()','tBodyAccMag-sma()','tGravityAccMag-mean()','tGravityAccMag-sma()']]

In [ ]:
for one_list in duplicate_columns.values():
    X_train.drop(columns=one_list,inplace=True)
    X_test.drop(columns=one_list,inplace=True)

In [ ]:


print(X_train.shape)
print(X_test.shape)



In [ ]:


from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=0.05)



In [ ]:
sel.fit(X_train)

In [ ]:
sum(sel.get_support())

In [ ]:
columns = X_train.columns[sel.get_support()]

In [ ]:
columns

In [ ]:
X_train = sel.transform(X_train)
X_test = sel.transform(X_test)

X_train = pd.DataFrame(X_train, columns=columns)
X_test = pd.DataFrame(X_test, columns=columns)

In [ ]:


print(X_train.shape)
print(X_test.shape)



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Use Rose Pine Dawn style
sns.set_theme(style="white", font_scale=0.5)  # Smaller font for high-dimension data

def big_corr_heatmap():
    corr_matrix = X_train.corr()

    # Set up a big figure so the plot can be zoomed
    fig, ax = plt.subplots(figsize=(50, 40))  # Adjust size if needed
    fig.patch.set_facecolor('#faf4ed')

    # Draw the heatmap
    sns.heatmap(corr_matrix,
                cmap="mako",  # a soft seaborn-compatible colormap
                linewidths=0.05,
                linecolor='#e0def4',
                square=True,
                cbar_kws={'shrink': 0.4},
                xticklabels=False,
                yticklabels=False,
                ax=ax)

    # Optional: Title
    ax.set_title("Correlation Heatmap of Features", fontsize=20, weight='bold', pad=20)

    # Tighter layout for clarity
    plt.tight_layout()
    plt.show()


In [ ]:
def big_corr_heatmap():
    corr_matrix = X_train.corr()

    fig, ax = plt.subplots(figsize=(50, 40))
    fig.patch.set_facecolor('#faf4ed')

    sns.heatmap(
        corr_matrix,
        cmap="mako",
        linewidths=0.05,
        linecolor='#e0def4',
        square=True,
        cbar_kws={'shrink': 0.4},
        xticklabels=False,
        yticklabels=False,
        ax=ax
    )

    ax.set_title("Correlation Heatmap of Features", fontsize=20, weight='bold', pad=20)

    plt.tight_layout()
    plt.show()

# Show it
big_corr_heatmap()


In [ ]:
corr_matrix = X_train.corr()

In [ ]:
columns = corr_matrix.columns

columns_to_drop = []

for i in range(len(columns)):
    for j in range(i + 1, len(columns)):
        if corr_matrix.loc[columns[i], columns[j]] > 0.95:
            columns_to_drop.append(columns[j])

print(len(columns_to_drop))

In [ ]:
columns_to_drop=set(columns_to_drop)
len(columns_to_drop)

In [ ]:
X_train.shape

In [ ]:


X_train.drop(columns = columns_to_drop, axis = 1, inplace=True)
X_test.drop(columns = columns_to_drop, axis = 1, inplace=True)



In [ ]:


print(X_train.shape)
print(X_test.shape)



In [ ]:
#ANOVA

In [ ]:


from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest


sel = SelectKBest(f_classif, k=153).fit(X_train, y_train)

X_train.columns[sel.get_support()]



In [ ]:
columns = X_train.columns[sel.get_support()]

In [ ]:


X_train = sel.transform(X_train)
X_test = sel.transform(X_test)

X_train = pd.DataFrame(X_train, columns=columns)
X_test = pd.DataFrame(X_test, columns=columns)



In [ ]:


print(X_train.shape)
print(X_test.shape)



In [ ]:
X_train.head()

In [ ]:
log_reg = LogisticRegression(max_iter=1000)  # Increase max_iter if it doesn't converge
log_reg.fit(X_train, y_train)

y_pred = log_reg.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy:", accuracy)

In [ ]:
from sklearn.svm import SVC

svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train, y_train)

y_pred = svm_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy (SVM):", accuracy)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy (Random Forest):", accuracy)


In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Decision Tree
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train, y_train)

y_pred = dt_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy (Decision Tree):", accuracy)


In [ ]:
pip install lazypredict

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier()
models = clf.fit(X_train, X_test, y_train, y_test)

print(models[0])

# **5. Other Filter Based Methods (not recommended)**

In [ ]:
df

In [ ]:
df

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Separate features and target
X = df.drop('Activity', axis=1)
y = df['Activity']

# Encode target labels
le = LabelEncoder()
y = le.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def get_duplicate_columns(df):

    duplicate_columns = {}
    seen_columns = {}

    for column in df.columns:
        current_column = df[column]

        # Convert column data to bytes
        try:
            current_column_hash = current_column.values.tobytes()
        except AttributeError:
            current_column_hash = current_column.to_string().encode()

        if current_column_hash in seen_columns:
            if seen_columns[current_column_hash] in duplicate_columns:
                duplicate_columns[seen_columns[current_column_hash]].append(column)
            else:
                duplicate_columns[seen_columns[current_column_hash]] = [column]
        else:
            seen_columns[current_column_hash] = column

    return duplicate_columns

In [ ]:
duplicate_columns = get_duplicate_columns(X_train)

In [ ]:
duplicate_columns

In [ ]:
X_train[['tBodyAccMag-mean()','tBodyAccMag-sma()','tGravityAccMag-mean()','tGravityAccMag-sma()']]

In [ ]:
for one_list in duplicate_columns.values():
    X_train.drop(columns=one_list,inplace=True)
    X_test.drop(columns=one_list,inplace=True)

In [ ]:


print(X_train.shape)
print(X_test.shape)



In [ ]:
#CHI2

In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


In [ ]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
pipeline = Pipeline([
    ('feature_selection', SelectKBest(score_func=chi2)),
    ('clf', RandomForestClassifier(random_state=42))
])


In [ ]:
param_grid = {
    'feature_selection__k': [10, 20, 30, 50, 75, 100, 110, 120]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)


In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy with top chi2-selected features: {accuracy:.4f}")

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier()
models = clf.fit(X_train, X_test, y_train, y_test)

print(models[0])

In [ ]:
#Mutual Info

In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


In [ ]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
pipeline = Pipeline([
    ('feature_selection', SelectKBest(score_func=mutual_info_classif)),
    ('clf', RandomForestClassifier(random_state=42))
])

In [ ]:
param_grid = {
    'feature_selection__k': [10, 20, 30, 50, 75, 'all']
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)


In [ ]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)

print("Best k value (Mutual Info):", grid_search.best_params_['feature_selection__k'])
print(f"Accuracy with best k (MI): {accuracy:.4f}")


In [ ]:
mi_scores = mutual_info_classif(X_train_scaled, y_train)
sorted_indices = np.argsort(mi_scores)[::-1]
print("Top 10 MI feature indices:", sorted_indices[:10])
print("Top 10 MI scores:", mi_scores[sorted_indices[:10]])


# **6.Iterative feature selection - CHI SQUARE**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [ ]:
num_features = X_train_scaled.shape[1]

k_values = []
accuracies = []

In [ ]:
for k in tqdm(range(1, num_features + 1)):
    selector = SelectKBest(score_func=chi2, k=k)
    X_train_k = selector.fit_transform(X_train_scaled, y_train)
    X_test_k = selector.transform(X_test_scaled)

    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_train_k, y_train)
    y_pred = clf.predict(X_test_k)
    acc = accuracy_score(y_test, y_pred)

    k_values.append(k)
    accuracies.append(acc)

In [ ]:
# prompt: k_values = np.array(k_values)
# accuracies = np.array(accuracies)
# display them and save them as a csv

import numpy as np
import pandas as pd

k_values = np.array(k_values)
accuracies = np.array(accuracies)

# Display k_values and accuracies
print("k_values:", k_values)
print("accuracies:", accuracies)

# Create a DataFrame
df_results = pd.DataFrame({'k_values': k_values, 'accuracies': accuracies})

# Save to CSV
df_results.to_csv('k_accuracy_results.csv', index=False)


In [ ]:
import matplotlib.pyplot as plt


def plot_elbow_curve(k_values, accuracies):
    plt.figure(figsize=(12, 6))
    plt.plot(
        k_values,
        accuracies,
        marker='o',
        linestyle='-',
        color='#eb6f92',
        linewidth=2.5,
        markersize=8,
        markerfacecolor='#f6c177',
        markeredgecolor='#31748f'
    )

    plt.title('Elbow Curve: Accuracy vs. Number of Features (Chi²)', fontsize=18, weight='bold')
    plt.xlabel('Number of Selected Features (k)', fontsize=14)
    plt.ylabel('Accuracy', fontsize=14)

    plt.grid(True, color='#c4a7e7', linestyle='--', linewidth=0.5)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    plt.tight_layout()
    plt.show()


In [ ]:
plot_elbow_curve(k_values, accuracies)

In [ ]:
# Get top 30 k-values
acc_array = np.array(accuracies)
k_array = np.array(k_values)
top_indices = acc_array.argsort()[::-1][:30]

top_k_acc = [(k_array[i], acc_array[i]) for i in top_indices]
print("Top 10 k-values with corresponding accuracies (Chi²):")
for k, acc in top_k_acc:
    print(f"k = {k}, Accuracy = {acc:.4f}")


So, the best accuracy we are getting is a t k = 180

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

# Fit Chi2 selector
selector_180 = SelectKBest(score_func=chi2, k=180)
X_train_180 = selector_180.fit_transform(X_train_scaled, y_train)
X_test_180 = selector_180.transform(X_test_scaled)

# Get selected feature indices
selected_indices_180 = selector_180.get_support(indices=True)
print("Selected feature indices (k=180):", selected_indices_180)


In [ ]:

if isinstance(X_train, pd.DataFrame):
    selected_features_180 = X_train.columns[selected_indices_180]
    print("Selected feature names (k=180):")
    for i, name in enumerate(selected_features_180, 1):
        print(f"{i:3}: {name}")


In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Step 1: Select top 180 features
selector_180 = SelectKBest(score_func=chi2, k=180)
X_train_180 = selector_180.fit_transform(X_train_scaled, y_train)
X_test_180 = selector_180.transform(X_test_scaled)

# Step 2: Train model
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_180, y_train)
y_pred = clf.predict(X_test_180)

# Step 3: Compute metrics
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
rec = recall_score(y_test, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
cm = confusion_matrix(y_test, y_pred)

# Display metrics
print("=== Performance with Top 180 Chi² Features ===")
print(f"Accuracy       : {acc:.4f}")
print(f"Precision      : {prec:.4f}")
print(f"Recall         : {rec:.4f}")
print(f"F1-Score       : {f1:.4f}")
print("\n=== Confusion Matrix ===")
print(cm)

# Optional: Detailed class-wise report
print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred, zero_division=0))


# **7. Conclusion**

## 📌 Conclusion

In this notebook, **filter-based feature selection methods** were used to improve the model's performance by reducing dimensionality and selecting the most informative features.

From the initial **Exploratory Data Analysis (EDA)**, it was clear that this is a **discrete vs categorical classification** problem. Based on this observation, I began the process with **Fisher’s Score-based feature selection**, which is well-suited for such scenarios.

---

### 🔍 Summary of Dataset & Base Model

- **Total Features**: 561  
- **Base Model**: Logistic Regression (used because it is:
  - Efficient for high-dimensional data
  - Easy to interpret
  - Provides strong baseline performance)
- **Accuracy with all 561 features**: **98.7%**

---

### ✅ Fisher's Score-Based Feature Selection + Similar Feature Elimination

1. **Data Preprocessing Steps**:
   - Removed duplicate and constant features
   - Eliminated highly similar features using cosine similarity-based filtering
   - Performed feature selection using Fisher Score

2. **Final Feature Count**: 153  
3. **Accuracy (Logistic Regression)**: **97.0%**  
4. **Highest Accuracy (using other classifiers)**: **99.0%** *(not recommended due to potential overfitting or model complexity)*

---

### 🔁 Iterative Chi² Feature Selection

- An **empty dictionary** was initialized to store `(k, accuracy)` for each iteration.
- Loop ran from `k = 1` to `k = 561` (i.e., `'all'`).
- For each `k`, the Chi² filter was applied and accuracy recorded.
- After collecting all results, the top 30 entries were analyzed.

#### ✅ Best `k` (Lowest `k` with Top Accuracy): **180**

- **Model Used**: Logistic Regression  
- **Accuracy with 180 Chi² Features**: **98.3%**  
- This is slightly below the base model but still very competitive, considering the dimensionality reduction.

---

### ⏱ Time Comparison of Methods

| Method                             | Time Taken     |
|------------------------------------|----------------|
| Fisher’s Score + Feature Elimination | ~5 minutes     |
| Iterative Chi² (1 to all)           | ~1 hour 10 mins |

---

### 📌 Final Thoughts

- **Fisher's Score** with feature elimination is **faster** and gives a **decent performance drop (1.7%)**.
- **Chi²** is **computationally heavier**, but selecting an optimal `k` gave us a **very close accuracy** to the full feature set.
- Both methods helped reduce overfitting risks, simplified the model, and made it easier to interpret.
